In [30]:
# Random Forrest for first pass
import numpy as np
import pandas as pd
import toms_tools as tt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

In [31]:
#read in data
df = pd.read_csv('data.csv')
df.drop("Unnamed: 0",1, inplace=True)

In [32]:
df.head()

,1stFlrSF,2ndFlrSF,FullBath,GarageArea,GrLivArea,LotFrontage,TotalBsmtSF,second_floor,HeatingQC_Ex,HeatingQC_Fa,...,OverallQual_6,OverallQual_7,OverallQual_8,OverallQual_9,OverallQual_10,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,SalePrice
0,0.109641,0.413559,0.50,0.368280,0.259231,0.207668,0.140098,1.0,1,0,...,0,1,0,0,0,0,0,1,0,208500.0
1,0.194917,0.000000,0.50,0.309140,0.174830,0.255591,0.206547,0.0,1,0,...,1,0,0,0,0,0,0,0,1,181500.0
2,0.123083,0.419370,0.50,0.408602,0.273549,0.217252,0.150573,1.0,1,0,...,0,1,0,0,0,0,0,1,0,223500.0
3,0.131695,0.366102,0.25,0.431452,0.260550,0.191693,0.123732,1.0,0,0,...,0,1,0,0,0,0,0,0,1,140000.0
4,0.170342,0.509927,0.50,0.561828,0.351168,0.268371,0.187398,1.0,1,0,...,0,0,1,0,0,0,0,1,0,250000.0


In [33]:
#split into train test sets
train = df.loc[df["SalePrice"] > 0]

In [34]:
test =  df.loc[df["SalePrice"].isnull() == True]

In [35]:
scaler = MinMaxScaler().fit(train["SalePrice"])
y_train = scaler.transform(train["SalePrice"])


c:\python\python35\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
c:\python\python35\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [36]:
x_train = train.drop("SalePrice",1).values

In [37]:
x_test = test.drop("SalePrice",1).values

Index(['1stFlrSF', '2ndFlrSF', 'BedroomAbvGr', 'BldgType', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFullBath', 'BsmtQual',
       'CentralAir', 'ExterQual', 'Exterior1st', 'Exterior2nd', 'Fireplaces',
       'Foundation', 'FullBath', 'GarageArea', 'GarageCars', 'GarageFinish',
       'GarageType', 'GrLivArea', 'HalfBath', 'HeatingQC', 'HouseStyle',
       'KitchenAbvGr', 'KitchenQual', 'LotFrontage', 'LotShape', 'MSSubClass',
       'MSZoning', 'MasVnrType', 'MoSold', 'Neighborhood', 'OverallCond',
       'OverallQual', 'RoofStyle', 'SaleCondition', 'TotRmsAbvGrd',
       'TotalBsmtSF', 'second_floor', 'Age', 'YearsSinceSale', 'SalePrice'],
      dtype='object')

In [38]:
# forest
forest = RandomForestRegressor( n_estimators=200,
                                max_depth=5,
                                min_samples_leaf = 100,
                                n_jobs=-1)
forest.fit(x_train, y_train)
preds = forest.predict(x_train)

In [40]:
#rescale
preds_rescaled = scaler.inverse_transform(preds)
y_train_rescaled = scaler.inverse_transform(y_train)


c:\python\python35\lib\site-packages\sklearn\preprocessing\data.py:374: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
c:\python\python35\lib\site-packages\sklearn\preprocessing\data.py:374: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [41]:
print(mean_squared_error(preds_rescaled, y_train_rescaled))

2193981782.29


In [43]:
np.sqrt(2193981782.29)

46839.959247313614

In [44]:
ans = forest.predict(x_test)

In [45]:
ans_rescale = scaler.inverse_transform(ans)

c:\python\python35\lib\site-packages\sklearn\preprocessing\data.py:374: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [46]:
df_ans = pd.DataFrame()

In [48]:
df_ans["Id"] = test.index

In [50]:
df_ans["SalePrice"] = ans_rescale

In [52]:
df_ans.drop('id',1, inplace=True)

In [54]:
df_ans.index = df_ans['Id']

In [56]:
df_ans.drop('Id', 1, inplace = True)

In [58]:
df_ans.shape

(1459, 1)

In [63]:
reindex = df_ans.index

In [65]:
reindex = reindex + 1

In [68]:
df_ans.index = reindex

In [69]:
df_ans.to_csv("sub.csv")